In [ ]:
!pip install sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer, InputExample, losses, evaluation, util , SentencesDataset
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import random
import tensorflow as tf
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv("df.csv")

In [8]:
n = 5000

In [9]:
all_q = []
all_des = []
all_title = []
all_index = []

for i in range(n):
    if("(Novel)" in df.loc[i, "title"] or "(Light Novel)" in df.loc[i, "title"] or "(Webcomic)" in df.loc[i, "title"] or "(Doujinshi)" in df.loc[i, "title"]):
        continue
    if(df.loc[i, "q"]):
        all_q.append(df.loc[i, "q"].split('\n')[0])
        all_des.append(str(df.loc[i, "description"]))
        all_title.append(str(df.loc[i, "title"]))
        all_index.append(i)

In [ ]:
np.save('all_title.npy', all_title)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(all_q, all_des, test_size=0.25, random_state=42)

In [11]:
x_tt, x_val, y_tt, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [ ]:
#for first time
model = SentenceTransformer('Madnesss/fine-tune-paraphrase-multilingual-mpnet-base-v2')

In [12]:
model = SentenceTransformer("/content/gdrive/MyDrive/model/model_multi_tripet_2.pt")

In [ ]:
# for cosine sim loss
# train_examples = []
# for i in range(len(x_train)):
#   train_examples.append(InputExample(texts=[str(x_train[i]), str(y_train[i])], label=1.0))

In [29]:
em_des = model.encode(y_train, convert_to_tensor=True)
em_q = model.encode(x_train, convert_to_tensor=True)
cosine_scoreee = util.cos_sim(em_des, em_des)

In [30]:
#generate negative example using tripet loss
train_examples = []
n_sam = len(x_train)
counter = 0

for i in range(n_sam):
  pivot_q = x_train[i]
  all_idx = torch.topk(cosine_scoreee[i], 2).indices
  if(all_idx[0]==i):
    train_examples.append(InputExample(texts=[pivot_q, y_train[i], y_train[all_idx[1]]]))
  else:
    train_examples.append(InputExample(texts=[pivot_q, y_train[i], y_train[all_idx[0]]]))

In [31]:
train_dataloader = DataLoader(train_examples, batch_size=16)
train_loss = losses.TripletLoss(model=model)

model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs= 3, warmup_steps=100, show_progress_bar=True)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/199 [00:00<?, ?it/s]

Iteration:   0%|          | 0/199 [00:00<?, ?it/s]

Iteration:   0%|          | 0/199 [00:00<?, ?it/s]

In [ ]:
model.save("./model_multi_tripet_2.pt")

In [ ]:
!zip -r /content/model_multi_tripet_2.zip /content/model_multi_tripet_2.pt

  adding: content/model_multi_tripet_2.pt/ (stored 0%)
  adding: content/model_multi_tripet_2.pt/sentence_bert_config.json (deflated 4%)
  adding: content/model_multi_tripet_2.pt/1_Pooling/ (stored 0%)
  adding: content/model_multi_tripet_2.pt/1_Pooling/config.json (deflated 47%)
  adding: content/model_multi_tripet_2.pt/config_sentence_transformers.json (deflated 26%)
  adding: content/model_multi_tripet_2.pt/sentencepiece.bpe.model (deflated 49%)
  adding: content/model_multi_tripet_2.pt/README.md (deflated 58%)
  adding: content/model_multi_tripet_2.pt/special_tokens_map.json (deflated 52%)
  adding: content/model_multi_tripet_2.pt/tokenizer_config.json (deflated 49%)
  adding: content/model_multi_tripet_2.pt/modules.json (deflated 53%)
  adding: content/model_multi_tripet_2.pt/pytorch_model.bin (deflated 9%)
  adding: content/model_multi_tripet_2.pt/config.json (deflated 48%)
  adding: content/model_multi_tripet_2.pt/tokenizer.json (deflated 76%)


In [ ]:
path = 'Madnesss/fine-tune-paraphrase-multilingual-mpnet-base-v2'
model = SentenceTransformer(path)

In [32]:
val_q = x_test

In [33]:
embeddings_des = model.encode(all_des, convert_to_tensor=True)

In [34]:
all_val_q = []
label = []
for i in range(len(val_q)):
    embedding_q = model.encode(val_q[i], convert_to_tensor=True)
    idxx = all_q.index(val_q[i])
    label.append(idxx)
    all_val_q.append(embedding_q)

In [ ]:
len(embeddings_des[0])

768

In [ ]:
len(all_val_q)

1060

In [35]:
acc = 0
n_val = len(all_val_q)
topk = 5

for i in range(n_val):
    cosine_scores = util.cos_sim(all_val_q[i], embeddings_des)
    #print(len(cosine_scores[0]))
    #max_val = max(cosine_scores[0])
    all_idx = torch.topk(cosine_scores[0], topk).indices
    ans = []
    for j in all_idx:
        ans.append(all_index[int(j)])

    #check idex in all_des==idx
    if(all_index[label[i]] in ans):
        acc += 1

In [ ]:
print("top1 acc :", acc/len(all_val_q))

top1 acc : 0.0


In [ ]:
print("top3 acc :", acc/len(all_val_q))

top3 acc : 0.0


In [36]:
print("top5 acc :", acc/len(all_val_q))

top5 acc : 0.6056603773584905
